# **Milestone 2: Data Ingestion Pipeline**

In [27]:
#data ingestion pipeline using pandas, checked for both raw and processed data
#raw data stored in input file path and processed data in the output file
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import os

# Config: input/output filenames (will try fallbacks if INPUT_FILE not found)
INPUT_FILE = "dynamic_pricing.csv"
OUTPUT_FILE = "cleaned_csv_data.csv"
FALLBACK_INPUTS = [INPUT_FILE, "./archive (6)/dynamic_pricing.csv", "dynamic_pricing_cleaned.csv"]

def find_existing_file(candidates):
    for p in candidates:
        if os.path.isfile(p):
            return p
    return None

def load_data(file_path):
    # allow passing None and try fallbacks
    path = file_path if file_path and os.path.isfile(file_path) else find_existing_file(FALLBACK_INPUTS)
    if path is None:
        raise FileNotFoundError(f"No input file found. Tried: {file_path} and fallbacks: {FALLBACK_INPUTS}")
    df = pd.read_csv(path)
    print("Data loaded from:", path, "Shape:", df.shape)
    return df

def normalize_column_names(df):
    df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')
    return df

def safe_label_encode(series, max_unique=200):
    # guard against huge-cardinality encoding that can be harmful
    s = series.fillna('missing').astype(str)
    uniques = s.nunique()
    if uniques > max_unique:
        print(f"Skipping label-encoding for column with high cardinality ({uniques} uniques).")
        return s
    return LabelEncoder().fit_transform(s)

def remove_outliers_iqr(df, col):
    # only remove outliers when there are enough distinct values
    if df[col].nunique() < 3:
        return df
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    if pd.isna(IQR) or IQR == 0:
        return df
    mask = ~((df[col] < (Q1 - 1.5 * IQR)) | (df[col] > (Q3 + 1.5 * IQR)))
    return df.loc[mask]

def clean_data(df, drop_na_threshold=0.0):
    # Normalize column names early
    df = normalize_column_names(df)

    # Optionally drop columns or rows with too many missing values (threshold on rows is 0 by default)
    if drop_na_threshold > 0:
        df = df.dropna(thresh=int((1 - drop_na_threshold) * len(df.columns)), axis=1)
    # drop rows with all-NaN
    df = df.dropna(how='all')

    # Separate column types
    numeric_cols = df.select_dtypes(include='number').columns.tolist()
    cat_cols = df.select_dtypes(include='object').columns.tolist()

    # Fill missing values: numeric -> median, categorical -> 'missing'
    for col in numeric_cols:
        median = df[col].median()
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col] = df[col].fillna(median)

    for col in cat_cols:
        df[col] = df[col].astype(str).fillna('missing').str.strip()

    # Remove outliers for numeric cols (safely)
    for col in numeric_cols:
        try:
            df = remove_outliers_iqr(df, col)
        except Exception as e:
            print(f"Skipping outlier removal on {col}: {e}")

    df = df.drop_duplicates()

    # Label-encode categorical columns (safely)
    for col in cat_cols:
        try:
            df[col] = safe_label_encode(df[col])
        except Exception as e:
            print(f"Could not label-encode {col}: {e}")

    print("Data cleaned successfully")
    print("Final shape after cleaning:", df.shape)
    return df

def run_pipeline(input_file=INPUT_FILE, output_file=OUTPUT_FILE):
    df_raw = load_data(input_file)
    df_cleaned = clean_data(df_raw)
    print("Pipeline completed successfully.")
    return df_cleaned

if __name__ == "__main__":
    df_cleaned = run_pipeline()


Data loaded from: ./archive (6)/dynamic_pricing.csv Shape: (1000, 10)
Data cleaned successfully
Final shape after cleaning: (990, 10)
Pipeline completed successfully.


In [30]:
# KPIs for Revenue lift, Profit Margin, Conversion Rate — robust version
import pandas as pd

def find_col(cols, candidates):
    cols_l = [c.lower() for c in cols]
    for cand in candidates:
        for i, c in enumerate(cols_l):
            if cand.lower() in c:
                return cols[i]
    return None

# load cleaned data if present, else try the pipeline output
if os.path.isfile('cleaned_csv_data.csv'):
    df = pd.read_csv('cleaned_csv_data.csv')
else:
    # try fallback names or run pipeline
    candidates = ['cleaned_csv_data.csv', 'baseline_threshold_pricing.csv', 'cleaned_dynamic_pricing.csv']
    found = None
    for p in candidates:
        if os.path.isfile(p):
            found = p
            break
    if found:
        df = pd.read_csv(found)
    else:
        print('No cleaned CSV found; please run the ingestion pipeline first (run_pipeline).')
        raise FileNotFoundError('cleaned_csv_data.csv not found')

# normalize columns
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# detect revenue-like and loyalty-like columns
revenue_col = find_col(df.columns, ['historical_cost_of_ride', 'historical_cost', 'cost_of_ride', 'cost', 'fare'])
loyalty_col = find_col(df.columns, ['customer_loyalty_status', 'customer_loyalty', 'loyalty_status', 'loyalty'])

if revenue_col is None:
    raise KeyError('No revenue-like column found. Available columns: ' + ','.join(df.columns))

# ensure numeric
df[revenue_col] = pd.to_numeric(df[revenue_col], errors='coerce').fillna(0)
df['revenue'] = df[revenue_col]
df['profit'] = df['revenue'] * 0.3
df['profit_margin'] = (df['profit'] / df['revenue']).replace([float('inf'), -float('inf')], 0).fillna(0) * 100

if loyalty_col is not None:
    # coerce loyalty to numeric percentile-style conversion if possible
    try:
        df[loyalty_col] = pd.to_numeric(df[loyalty_col], errors='coerce')
        df['conversion_rate'] = (df[loyalty_col] / df[loyalty_col].max()).fillna(0) * 100
        
    except Exception:
        # fallback: treat loyalty as categorical frequency
        df['conversion_rate'] = 0
else:
    df['conversion_rate'] = 0

baseline_revenue = df['revenue'].iloc[0] if len(df) > 0 else 0
df['revenue_lift_pct'] = ((df['revenue'] - baseline_revenue) / baseline_revenue).replace([float('inf'), -float('inf')], 0).fillna(0) * 100 if baseline_revenue != 0 else 0

kpi_summary = df[['revenue', 'profit_margin', 'conversion_rate']].copy()
kpi_summary['revenue_lift_pct'] = df['revenue_lift_pct'] if isinstance(df['revenue_lift_pct'], (pd.Series, list)) else [df['revenue_lift_pct']]
print(kpi_summary.head())

      revenue  profit_margin  conversion_rate  revenue_lift_pct
0  284.257273           30.0              0.0          0.000000
1  173.874753           30.0              0.0        -38.831907
2  329.795469           30.0              0.0         16.020064
3  470.201232           30.0              0.0         65.413967
4  579.681422           30.0              0.0        103.928440
